# Han

## Index
1. [Define required clock parameters](#Define-required-clock-parameters)
2. [Download necessary data](#Download-necessary-data)
3. [Load data](#-Load-data)
4. [Extract features and weights](#Extract-features-and-weights)
5. [Load weights into pyaging model](#Load-weights-into-pyaging-model)
6. [Add reference values](#Add-reference-values)
7. [Add preprocessing and postprocesssing steps](#Add-preprocessing-and-postprocesssing-steps)
8. [Check all data objects](#Check-all-data-objects)
9. [Write clock dictionary](#Write-clock-dictionary)
10. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import marshal
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Define required clock parameters

Let's define some required information first:

In [2]:
clock_name = 'han'
data_type = 'methylation'
model_class = 'LinearModel'
species = 'Homo sapiens'
year = 2020
approved_by_author = '⌛'
citation = "Han, Yang, et al. \"New targeted approaches for epigenetic age predictions.\" BMC biology 18 (2020): 1-15."
doi = "https://doi.org/10.1186/s12915-020-00807-2"
notes = None

## Download necessary data

In [3]:
# No need to download; see below

## Load data

In [4]:
# from authors
cpg_sites = [
    "(Intercept)",
    'cg19283806',
    'cg11807280',
    'cg00329615',
    'cg22454769',
    'cg16867657',
    'cg22796704',
    'cg09809672',
    'cg18618815',
    'cg25533247',
    'cg02286081',
    'cg20222376',
    'cg19344626',
    'cg07082267',
    'cg15845821',
    'cg11741201',
    'cg16054275',
    'cg18933331',
    'cg20249566',
    'cg16604658',
    'cg07583137',
    'cg16008966',
    'cg14556683',
    'cg03746976',
    'cg14314729',
    'cg03431918',
    'cg22156456',
    'cg23078123',
    'cg09748749',
    'cg17457912',
    'cg06492796',
    'cg17593342',
    'cg05308819',
    'cg22512670',
    'cg01820962',
    'cg06639320',
    'cg03224418',
    'cg17436656',
    'cg19500607',
    'cg03735592',
    'cg20669012',
    'cg19761273',
    'cg07080372',
    'cg03638795',
    'cg19722847',
    'cg24711336',
    'cg26935102',
    'cg10221746',
    'cg02085953',
    'cg04604946',
    'cg08558886',
    'cg22361181',
    'cg04208403',
    'cg12623930',
    'cg21572722',
    'cg17885226',
    'cg00748589',
    'cg13033938',
    'cg19784428',
    'cg22016779',
    'cg01974375',
    'cg25256723',
    'cg24724428',
    'cg07547549',
    'cg25410668',
    'cg21296230'
]

coefficients = [
    0.711184864,
    -0.588354066,
    -0.212038592,
    0.014351188,
    0.051285529,
    2.152191741,
    -0.689940565,
    -0.643729974,
    -0.772516118,
    0.116662569,
    -0.233409678,
    0.002802259,
    -0.062172432,
    -0.224027294,
    1.535209377,
    0.344367661,
    0.188826525,
    -0.409150014,
    -0.776065004,
    0.500336643,
    0.06125005,
    -0.391624093,
    0.100449175,
    0.02000403,
    0.266044453,
    -0.259829677,
    0.254063071,
    -0.726178338,
    -1.141947121,
    -0.06322441,
    -0.196926134,
    0.85613244,
    -0.887977059,
    -0.334654336,
    -0.854110638,
    1.916122401,
    0.92208575,
    -0.070665617,
    0.524707402,
    0.319375235,
    0.376055859,
    0.033361038,
    -1.458360975,
    -0.267930475,
    -0.590085273,
    0.642506165,
    0.470352872,
    0.273581649,
    -0.637989789,
    -1.109388991,
    -0.16886654,
    0.662451226,
    -0.091891613,
    0.086290028,
    -0.426089316,
    0.32615363,
    2.535639458,
    -3.626802894,
    0.097619541,
    -0.427604263,
    -0.41418774,
    -0.27412342,
    0.703772384,
    -0.110027226,
    0.283649813,
    0.928585964
]

## Extract features and weights

First, let's extract the features and weights:

In [5]:
df = pd.DataFrame({
    'feature': cpg_sites,
    'coefficient': coefficients
})

df.head()

,feature,coefficient
0,(Intercept),0.711185
1,cg19283806,-0.588354
2,cg11807280,-0.212039
3,cg00329615,0.014351
4,cg22454769,0.051286


Then, let's create lists for features and weights. Be careful about the intercept, as it usually shows up as a feature name.

In [6]:
features = df['feature'][1:].tolist()
weights = torch.tensor(df['coefficient'][1:].tolist()).unsqueeze(0)
intercept = torch.tensor([df['coefficient'][0]])

## Load weights into pyaging model

#### Linear model

In [7]:
model = pya.models.LinearModel(input_dim=len(features))

model.linear.weight.data = weights.float()
model.linear.bias.data = intercept.float()

model

LinearModel(
  (linear): Linear(in_features=65, out_features=1, bias=True)
)

## Add reference values

Some clocks have reference values in the case of missing features. It is also possible that these values are for preprocessing features rather than the clock features. Let's add a dictionary with the feature names as the keys.

In [8]:
reference_feature_values = None

## Add preprocessing and postprocesssing steps

The preprocessing and postprocessing objects are dictionaries with the following format, with all items required. It takes in x in the form of a numpy array.

In [9]:
preprocessing = None

Similarly is the case of postprocessing. Remember that your function must be compatible with torch and is applied to each number individually.

In [10]:
def postprocessing_function(x):
    """
    Applies an anti-logarithmic linear transformation to a PyTorch tensor.
    """
    adult_age=20
    
    if x < 0:
        # Exponential transformation for negative values
        age_tensor = (1 + adult_age) * math.exp(x) - 1
    else:
        # Linear transformation for non-negative values
        age_tensor = (1 + adult_age) * x + adult_age
    return age_tensor
    
postprocessing_function_string = marshal.dumps(postprocessing_function.__code__)

postprocessing_helper_objects = None

postprocessing = {
    'name': 'anti_log_linear',
    'postprocessing_function': postprocessing_function_string,
    'postprocessing_helper_objects': postprocessing_helper_objects
}

## Check all data objects

Let's print all data objects to check if they make sense.

#### features

In [11]:
def my_print_function():
    print(f"There are {len(features)} features.")
    print(features)
pya.utils.print_to_scrollable_output(my_print_function)

#### reference_feature_values

In [12]:
def my_print_function():
    if reference_feature_values:
        print(f"There are {len(reference_feature_values)} reference feature values.")
    print(reference_feature_values)
pya.utils.print_to_scrollable_output(my_print_function)

#### preprocessing

In [13]:
def my_print_function():
    print(preprocessing)
    if preprocessing:
        print(preprocessing_helper_objects)
pya.utils.print_to_scrollable_output(my_print_function)

#### postprocessing

In [14]:
def my_print_function():
    print(postprocessing)
    if postprocessing:
        print(postprocessing)
pya.utils.print_to_scrollable_output(my_print_function)

#### weight_dict

In [15]:
def my_print_function():
    for name, param in model.named_parameters():
        print(f"Layer: {name}")
        print(f"Shape: {param.shape}")
        print(param.data)
pya.utils.print_to_scrollable_output(my_print_function)

## Write clock dictionary

Let's put everything together and save:

In [16]:
clock_dict = {
    # Metadata
    'clock_name': clock_name,
    'data_type': data_type,
    'model_class': model_class,
    'species': species,
    'year': year,
    'approved_by_author': approved_by_author,
    'citation': citation,
    'doi': doi,
    "notes": notes,

    # Data
    'reference_feature_values': reference_feature_values if reference_feature_values else None,
    'preprocessing': preprocessing if preprocessing else None, 
    'features': features,
    'weight_dict': model.state_dict(),
    'postprocessing': postprocessing if postprocessing else None,
}

torch.save(clock_dict, f'../weights/{clock_name}.pt')

## Clear directory

Delete all files that are not clock jupyter notebooks:

In [17]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted folder: .ipynb_checkpoints
